In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from difflib import SequenceMatcher

In [2]:
categoryList = ['STORAGE', 'HANDLING', 'ACCESSORIAL']
facilityList = ['804- GARDEN CITY', '825-NORTHAMPTON','Alessandro','BolingBrook','Charleston','COPPELL','COR1','Fontana','FOREST','GARDENA','GOODYEAR','GOURGAR','Grand Prairie','GREENWOOD','Hayward','Hazelton','Houston','Indiana','Innovation','Joliet','Kansas','KENT','Lakewood','Marlay','Morgan Lakes','Murphy','NAVIGATION','New Jersey','Ontario','QUA','Quality-4400','Red Bluff','Reverse Service','Reyes','ROANOKE','Sacramento','Savannah','Seabrook','TACOMA','TOLLESON','Tucker','Turnbull','Valley','Valley View','Valley View B','Valley View C','Via Baron','WALB','Walnut','Willow']

In [3]:
def facilityMatcher(givenF):
    highestratio = 0
    facName = ''
    while (True):
        for f in facilityList:
            ratio = SequenceMatcher(None, f, givenF).ratio()
            if ratio > highestratio:
                highestratio = ratio
                facName = f
        if highestratio < 0.5:
            givenF = input("Input new Facility name:")
        else:
            return facName

In [4]:
billTo = input("Billing To:")
facility = input("Facility:")
facility = facilityMatcher(facility)
periodStart = input("Period Start:")
periodEnd = input("Period End:")

In [5]:
#Getting BNP
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
actions = ActionChains(driver)
driver.get("http://bnp.unisco.com/")
original_window = driver.current_window_handle
assert len(driver.window_handles) == 1

In [6]:
#Logging into BNP
interactor = driver.find_element(By.ID,"inputUserName")
interactor.send_keys("kevinnguyen")
interactor = driver.find_element(By.ID,"inputPassword")
interactor.send_keys("kevinn")
interactor = driver.find_element(By.XPATH,"/html/body/div/footer/div/button")
interactor.click()

In [7]:
#Clicking Sales Module from Module Dropdown Menu
select = driver.find_element(By.ID, 'TS_span_menu')
select.click()
time.sleep(0.5)
select = driver.find_element(By.XPATH, '//*[@id="headmenu_mn_active"]/div/ul/li[1]')
select.click()

In [8]:
#Clicking Invoice Management from Invoice Dropdown Menu
select = driver.find_element(By.XPATH, '//*[@id="headmenu"]/li[3]/span')
select.click()
time.sleep(0.5)
select = driver.find_element(By.XPATH, '//*[@id="headmenu_mn_active"]/div/ul/li[1]')
select.click()

In [9]:
#Inputting information on the Invoice Management Page and Searching

#Bill To
interactor = driver.find_element(By.XPATH, '//*[@id="sitecontent"]/div[2]/div[1]/div[6]/span/span/input')
interactor.send_keys(billTo)
time.sleep(0.5)
interactor.send_keys(Keys.ENTER)

#Facility
select = Select(driver.find_element(By.ID, 'ddlFacility'))
select.select_by_visible_text(facility)

#Invoice Status
interactor = driver.find_element(By.XPATH, '//*[@id="sitecontent"]/div[2]/div[2]/div[5]/div/div/input')
interactor.send_keys('Check')
time.sleep(0.5)
interactor.send_keys(Keys.ENTER)

#Period Start
interactor = driver.find_element(By.ID, 'inputPeriodStart')
interactor.send_keys(periodStart)

#Period End
interactor = driver.find_element(By.ID, 'inputPeriodEnd')
interactor.send_keys(periodEnd)

#Clicking Search
interactor = driver.find_element(By.XPATH, '//*[@id="sitecontent"]/div[2]/div[3]/div[11]/button')
interactor.click()

In [10]:
#Expanding Table to 50 items per page
interactor = driver.find_element(By.XPATH, '//*[@id="invoicegrid"]/div[4]/span[1]/span/span/span[1]')
interactor.click()
time.sleep(0.5)
interactor = driver.find_element(By.XPATH, '/html/body/div[25]/div/div[2]/ul/li[3]')
interactor.click()

time.sleep(0.5)
table = driver.find_element(By.XPATH,'//*[@id="invoicegrid"]/div[3]/table')
showTable = table.is_displayed
if showTable:
    rows = table.find_elements(By.TAG_NAME,"tr")
    
rowsList = []
for count, row in enumerate(rows):
    col = row.find_elements(By.TAG_NAME, "td")
    for cat in categoryList:
        if(cat in col[5].text):
            rowsList.append(count)
            break

for index in range(len(rowsList) - 1, -1, -1):
    num = str(rowsList[index] + 1)
    interactor = driver.find_element(By.XPATH, '//*[@id="invoicegrid"]/div[3]/table/tbody/tr[' + num +']/td[1]/label')
    actions.move_to_element(interactor).perform()
    interactor.click()

interactor = driver.find_element(By.XPATH, '//*[@id="btnExportInvoiceDetail"]')
interactor.click()